In [1]:
%load_ext autoreload

In [43]:
%autoreload 2
import sys
from pathlib import Path

parent = str(Path().absolute().parent)
if parent not in sys.path:
    sys.path.insert(0,parent)
from sherlock.evidence_store import EvidenceStore
from sherlock.models import Claim, Evidence, Argument, EvidenceCollection
from sherlock.agents import ClaimInvesgiationAgent
from sherlock.utils import export_argdown
import pyperclip

In [5]:
store = EvidenceStore("wally_evidence")

# Add our evidence
evidence_ids = []
evidence_ids.append(store.add_evidence("Librarian Jane saw someone matching Wally's description at 3pm"))
evidence_ids.append(store.add_evidence("Security camera shows someone in red and white stripes in the park"))
evidence_ids.append(store.add_evidence("Library card records don't show Wally checking out any books today"))
evidence_ids.append(store.add_evidence("Wally's phone GPS shows he was at the library coordinates"))



In [22]:


agent_pro = ClaimInvesgiationAgent(store, supports=True)
agent_con = ClaimInvesgiationAgent(store, supports=False)
# Create a test claim
wally_claim = Claim(text="Wally is in the library")


In [23]:
wally_claim = agent_pro.evaluate_claim(wally_claim)
wally_claim = agent_con.evaluate_claim(wally_claim)
# Evaluate it
wally_claim.likelihood

17:10:18 | INFO | Starting evaluation of claim for supporting evidence: Wally is in the library
17:10:18 | INFO | Starting iteration 1
17:10:30 | INFO | Tool_use stop reason. Response.content: [TextBlock(citations=None, text='<thinking>\nThe claim is "Wally is in the library". To evaluate this claim and find supporting evidence, I will:\n\n1. Analyze the key concepts in the claim:\n- Wally (a person)\n- library (a location)\n\n2. Search for evidence related to those key concepts using the query_evidence tool. Some potential search queries:\n- "Wally library"\n- "Wally in the library" \n- "Wally\'s location library"\n\n3. Evaluate the returned evidence to see if it supports the claim that Wally is in the library. Keep any evidence that indicates Wally\'s location is the library.\n\n4. If 2-3 pieces of supporting evidence are found, use the create_argument tool to form an argument that the claim is true, using the evidence.\n\n5. If insufficient supporting evidence is found after 3 searc

Likelihood(supporting=2, opposing=1)

In [61]:
wally_claim.model_dump()

{'text': 'Wally is in the library',
 'id': 'wally-is-in-the-library',
 'arguments': [{'id': 'd744b708-fa82-48cd-9522-87ffb71c22cb',
   'text': 'Based on visual identification, Wally is at the library',
   'supports': True,
   'evidence_collection': {'evidence': []},
   'subclaims': []}],
 'likelihood': {'supporting': 0, 'opposing': 0}}

In [59]:
arg = export_argdown(wally_claim)

In [60]:
with open('argdown.txt', 'w') as f:
    f.write(arg)

In [62]:

# 3. Create a claim
wally_claim = Claim(text="Wally is in the library")
print(f"Created claim: {wally_claim.text} (ID: {wally_claim.id})")

# 4. Query for relevant evidence
query_results = store.query("Wally library location", n_results=3)
print("\nRelevant evidence for query 'Wally library location':")
for i, result in enumerate(query_results):
    print(f"{i+1}. {result['text']} (ID: {result['id']})")

selected_evidence_id = query_results[1]['id']

# Create an argument manually
argument = Argument(
    text="Based on visual identification, Wally is at the library",
    supports=True,  # This supports our claim
    evidence_ids=[selected_evidence_id]
)

# Add to our claim
wally_claim.arguments.append(argument)
wally_claim._update_likelihood()



Created claim: Wally is in the library (ID: wally-is-in-the-library)

Relevant evidence for query 'Wally library location':
1. Wally's phone GPS shows he was at the library coordinates (ID: ev_4)
2. Wally's phone GPS shows he was at the library coordinates (ID: ev_8)
3. Librarian Jane saw someone matching Wally's description at 3pm (ID: ev_5)


In [63]:
argument.model_dump()

{'id': '1611f701-9993-4dfc-8ca1-b2832ca08a03',
 'text': 'Based on visual identification, Wally is at the library',
 'supports': True,
 'evidence_collection': {'evidence': []},
 'subclaims': []}

In [5]:
store.collection.get('ev_1')

{'ids': ['ev_1'],
 'embeddings': None,
 'documents': ["Librarian Jane saw someone matching Wally's description at 3pm"],
 'uris': None,
 'data': None,
 'metadatas': [{'type': 'evidence'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [39]:
wally_claim.model_dump()

{'text': 'Wally is in the library',
 'id': 'wally-is-in-the-library',
 'arguments': [{'id': '4f949f2b-38cd-4682-8838-9f44fd965e90',
   'text': "The evidence strongly supports that Wally was in the library. His phone's GPS coordinates place him at the library location. Additionally, a librarian reports seeing someone matching Wally's description in the library. Together, this evidence corroborates the claim that Wally was in fact in the library.",
   'supports': True,
   'evidence_collection': {'evidence': [{'id': 'ev_4',
      'text': "Wally's phone GPS shows he was at the library coordinates"},
     {'id': 'ev_5',
      'text': "Librarian Jane saw someone matching Wally's description at 3pm"}]},
   'subclaims': []},
  {'id': 'a4953eb5-0edb-4b2c-98f3-d7c078753fec',
   'text': 'There is limited evidence to suggest Wally may not currently be in the library. Library records do not show him checking out any books today. However, this alone does not prove he is not at the library, as he cou

In [64]:


# Create sample data
claim = Claim(text="Wally is in the library")

# Supporting evidence
ev4 = Evidence(id="ev_4", text="Wally's phone GPS shows he was at the library coordinates")
ev5 = Evidence(id="ev_5", text="Librarian Jane saw someone matching Wally's description at 3pm")
supporting_evidence = EvidenceCollection(evidence=[ev4, ev5])

# Opposing evidence
ev3 = Evidence(id="ev_3", text="Library card records don't show Wally checking out any books today")
opposing_evidence = EvidenceCollection(evidence=[ev3])

# Supporting argument
supporting_arg = Argument(
    text="The evidence strongly supports that Wally was in the library...",
    supports=True,
    evidence_collection=supporting_evidence
)

# Opposing argument
opposing_arg = Argument(
    text="There is limited evidence to suggest Wally may not currently be in the library...",
    supports=False,
    evidence_collection=opposing_evidence
)

# Add arguments to claim
claim.add_argument(supporting_arg)
claim.add_argument(opposing_arg)

# Export to Argdown
argdown_text = export_argdown(claim)
pyperclip.copy(argdown_text)